**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [91]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [92]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [93]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [94]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [95]:
df_supply

,0
0,822
1,381
2,144
3,936
4,633
...,...
95,246
96,414
97,767
98,381


In [96]:
df_demand

,0
0,40480
1,42532
2,21103
3,50497
4,4699
...,...
95,348
96,22518
97,8047
98,9620


In [97]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,95.644408,140.441453,75.457771,97.374822,84.175512,73.408446,127.101345,101.372091,94.331365,67.577038,...,95.848547,127.378491,73.066939,131.705174,77.816276,133.178833,85.083902,81.863887,94.874064,95.014273
1,95.746465,139.626015,75.060637,95.994133,83.656746,74.417202,125.445918,100.984846,93.054135,68.788836,...,94.794565,126.365666,72.955965,130.811229,77.281325,131.584315,86.362807,83.191692,95.348456,94.922217
2,108.446484,148.774621,86.329601,101.815457,94.408678,88.239093,128.608255,112.164907,99.664461,82.511148,...,102.835698,134.470267,85.161494,139.572284,88.009309,135.315927,100.015487,96.780428,108.781293,107.103598
3,97.199572,142.386157,77.269233,99.374556,86.032552,74.269132,129.040570,103.175191,96.336078,68.230199,...,97.839706,129.363241,74.746719,133.668198,79.680161,135.139016,85.650826,82.374986,96.181162,96.678668
4,107.829080,148.143498,85.691465,101.233481,93.768737,87.735165,128.096064,111.525854,99.066749,82.046933,...,102.215459,133.851261,84.533922,138.945054,87.369377,134.788961,99.567063,96.343519,108.197981,106.474560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,106.327795,147.863548,84.719962,101.750943,92.973768,85.344216,129.216816,110.606300,99.397095,79.494724,...,102.196297,133.844280,83.225381,138.766866,86.574786,135.797779,96.959076,93.699639,106.305763,105.190235
96,97.305391,141.525585,76.842655,97.911895,85.480290,75.328061,127.281986,102.764975,94.983552,69.503013,...,96.723937,128.292167,74.619890,132.723737,79.109443,133.445851,87.006023,83.781354,96.681316,96.579054
97,111.553166,150.224664,88.790198,102.147814,96.624370,92.357566,127.958493,114.524464,100.282353,86.782118,...,103.961224,135.539928,88.053220,140.884338,90.234610,134.838902,104.332269,101.127721,112.354297,109.940493
98,103.003845,145.476519,81.772680,100.216725,90.161810,81.696100,128.385220,107.684279,97.657819,75.858901,...,100.088409,131.727816,80.033433,136.480000,83.768061,134.820390,93.338164,90.091509,102.781539,102.004486


In [98]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [99]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

54383 2577921


In [100]:
m = gp.Model("probrema")

In [101]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [102]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [103]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [104]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [105]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [106]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [107]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 900 rows, 40000 columns and 140000 nonzeros
Model fingerprint: 0x901d397a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+05]


Presolve removed 300 rows and 0 columns
Presolve time: 0.17s
Presolved: 600 rows, 40000 columns, 110000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.020e+04
 Factor NZ  : 1.016e+05 (roughly 17 MB of memory)
 Factor Ops : 2.372e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.16010952e+11 -2.24398028e+10  1.26e+07 4.31e+01  3.18e+06     0s
   1   8.96478361e+09 -1.07018813e+10  9.92e+05 1.43e+01  3.47e+05     0s
   2   1.06775080e+09 -3.86588167e+09  1.21e+05 4.85e+00  7.34e+04     1s
   3   1.20557814e+08 -1.42476913e+09  1.36e+04 1.73e+00  2.08e+04     1s
   4   1.76456517e+07 -4.85574126e+08  1.72e+03 5.09e-01  6.47e+03     1s
   5   1.35510816e+07 -4.54774160e+08  1.25e+03 4.50e-01  6.00e+03     1s

Barrier performed 5 iterations in 0.65 secon

In [108]:
m.write("out.sol")